In [ ]:
### Libs ###

import numpy as np
import pandas as pd
import requests 
import io
from matplotlib import pyplot as plt
import matplotlib

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [ ]:
#Personal Smoking Habits
questionnaire_url_18 = 'https://wwwn.cdc.gov//Nchs/Nhanes/2017-2018/P_SMQ.XPT'
questionnaire_url_16 = 'https://wwwn.cdc.gov//Nchs/Nhanes/2015-2016/SMQ_I.XPT'
questionnaire_url_14 = 'https://wwwn.cdc.gov//Nchs/Nhanes/2013-2014/SMQ_H.XPT'
questionnaire_url_12 = 'https://wwwn.cdc.gov//Nchs/Nhanes/2011-2012/SMQ_G.XPT'
questionnaire_url_10 = 'https://wwwn.cdc.gov//Nchs/Nhanes/2009-2010/SMQ_F.XPT'
questionnaire_url_8  = 'https://wwwn.cdc.gov//Nchs/Nhanes/2007-2008/SMQ_E.XPT'
questionnaire_url_6  = 'https://wwwn.cdc.gov//Nchs/Nhanes/2005-2006/SMQ_D.XPT'
questionnaire_url_4  = 'https://wwwn.cdc.gov//Nchs/Nhanes/2003-2004/SMQ_C.XPT'
questionnaire_url_2  = 'https://wwwn.cdc.gov//Nchs/Nhanes/2001-2002/SMQ_B.XPT'
questionnaire_url_0  = 'https://wwwn.cdc.gov//Nchs/Nhanes/1999-2000/SMQ.XPT'

#Second Hand smoke Exposure
shs_questionnaire_url_18 = 'https://wwwn.cdc.gov//Nchs/Nhanes/2017-2018/P_SMQFAM.XPT'
shs_questionnaire_url_16 = 'https://wwwn.cdc.gov//Nchs/Nhanes/2015-2016/SMQFAM_I.XPT'
shs_questionnaire_url_14 = 'https://wwwn.cdc.gov//Nchs/Nhanes/2013-2014/SMQFAM_H.XPT'
shs_questionnaire_url_12 = 'https://wwwn.cdc.gov//Nchs/Nhanes/2011-2012/SMQFAM_G.XPT'
shs_questionnaire_url_10 = 'https://wwwn.cdc.gov//Nchs/Nhanes/2009-2010/SMQFAM_F.XPT'
shs_questionnaire_url_8  = 'https://wwwn.cdc.gov//Nchs/Nhanes/2007-2008/SMQFAM_E.XPT'
shs_questionnaire_url_6  = 'https://wwwn.cdc.gov//Nchs/Nhanes/2005-2006/SMQFAM_D.XPT'
shs_questionnaire_url_4  = 'https://wwwn.cdc.gov//Nchs/Nhanes/2003-2004/SMQFAM_C.XPT'
shs_questionnaire_url_2  = 'https://wwwn.cdc.gov//Nchs/Nhanes/2001-2002/SMQFAM_B.XPT'
shs_questionnaire_url_0  = 'https://wwwn.cdc.gov//Nchs/Nhanes/1999-2000/SMQFAM.XPT'


#VOC levels
voc_url_18 = 'https://wwwn.cdc.gov//Nchs/Nhanes/2017-2018/P_VOCWB.XPT'
voc_url_16 = 'https://wwwn.cdc.gov//Nchs/Nhanes/2015-2016/VOCWB_I.XPT'
voc_url_14 = 'https://wwwn.cdc.gov//Nchs/Nhanes/2013-2014/VOCWB_H.XPT'
voc_url_12 = 'https://wwwn.cdc.gov//Nchs/Nhanes/2011-2012/VOCWB_G.XPT'
voc_url_10 = 'https://wwwn.cdc.gov//Nchs/Nhanes/2009-2010/VOCWB_F.XPT'
voc_url_8  = 'https://wwwn.cdc.gov//Nchs/Nhanes/2007-2008/VOCWB_E.XPT'
voc_url_6  = 'https://wwwn.cdc.gov//Nchs/Nhanes/2005-2006/VOCWB_D.XPT'
voc_url_4  = 'https://wwwn.cdc.gov//Nchs/Nhanes/2003-2004/L04VOC_C.XPT'
voc_url_2  = 'https://wwwn.cdc.gov//Nchs/Nhanes/2001-2002/L04VOC_B.XPT'
voc_url_0  = 'https://wwwn.cdc.gov//Nchs/Nhanes/1999-2000/LAB04.XPT'

urls_2013 = [(shs_questionnaire_url_0, questionnaire_url_0, voc_url_0),(shs_questionnaire_url_2, questionnaire_url_2, voc_url_2),(shs_questionnaire_url_4, questionnaire_url_4, voc_url_4),
        (shs_questionnaire_url_6, questionnaire_url_6, voc_url_6),(shs_questionnaire_url_8, questionnaire_url_8, voc_url_8),(shs_questionnaire_url_10, questionnaire_url_10, voc_url_10),
        (shs_questionnaire_url_12, questionnaire_url_12, voc_url_12)]

urls_2014 = [(shs_questionnaire_url_14, questionnaire_url_14, voc_url_14),(shs_questionnaire_url_16, questionnaire_url_16, voc_url_16),
        (shs_questionnaire_url_18,questionnaire_url_18, voc_url_18)]


def read_sas_url(url):
    r = requests.get(url)
    return pd.read_sas(io.BytesIO(r.content), format= 'xport')

In [ ]:
def request_data(urls, predictors_list, response_var_d, response_var_sh):

    """
    This function requests a list of similar datasets from the cdc website

    urls: The url to the dataset
    predictor_list : the lsit of variables to be pulled from the dataset
    response_var_d: the response variable to be pulled from the direct smoking habits questionaire
    response_var_sh: the response variable to be pulled from the second hand smoke exposure questionaire
    """
    all_dfs = []

    for url_sh, url_d, url_X in urls:

        print(f"Fetching Data for Year: {url_X[34:43] }")
        
        X = read_sas_url(url_X)
        d = read_sas_url(url_d)
        sh = read_sas_url(url_sh)

        Data_year = pd.merge(pd.merge(X, d, on = ['SEQN']), sh, on = ['SEQN'])[predictors_list + [response_var_d, response_var_sh]]

        all_dfs.append(Data_year)
    
    Data_Xy = pd.concat(all_dfs)
    Data_Xy= Data_Xy.dropna().reindex()
    Data_Xy[response_var_d] = Data_Xy[response_var_d].astype(int)-1 
    Data_Xy[response_var_sh] = Data_Xy[response_var_sh].astype(int)

    return Data_Xy

def fetch_data(urls1, urls2, predictors_list, response1, response13, response14, save = False):

    """
    This function requests a Multiplte datasets from the cdc website: 
    this function is necessariy because datasets can have different variables/names with the same data
    Functionally, it just calls request_data and combines the frames.
    """

    df1 = request_data(urls1, predictors_list, response1, response13)
    df2 = request_data(urls2, predictors_list, response1, response14)
    Data_Xy = pd.concat([df1, df2])

    if save == True:
        #You have to put the path you want to save the frame to here
        Data_Xy.to_csv('/Users/omarafifi/Downloads/NHANES Project/data/NHANES.csv')
        print('Data Copied into Local Directory.')
        
    print("Data Load Complete.")

    return Data_Xy

def append_match(datasets, keep_cols):
    """ 
    This function takes a list of datasets and concats
    them while matching the column names. 
    """

    if not datasets or len(datasets) == 0:
        return None

    # Reove columns not in list for each data set
    filtered_datasets = [df[keep_cols] for df in datasets]

    # Extract common col names
    common_cols = filtered_datasets[0].columns

    # Check if all datasets have same cols
    for df in filtered_datasets[1:]:
        common_cols = common_cols.intersection(df.columns)

    # Concat all the datasets if assumptions are met
    append_df = pd.concat([df[common_cols] for df in filtered_datasets], ignore_index=True)

    return append_df



In [ ]:
#some of the respone formatting is inconsistent ... this reformats things. 
def relable(df):

    """
    The response variables for second hand smokers and direct smokers are different [0,1,2] vs [1,0]. 
    This function converts them so that they index from 0. 
    """

    data = df.copy()

    data['SMD460']+= 1
    data['SMD460'] = data['SMD460'].fillna(0)
    data = data[data['SMD460'] < 10]

    data['SMD460'] = (data['SMD460'] > 1).astype(int)


    data['SMD410'] = data['SMD410'].fillna(0)
    data = data[data['SMD410'] < 3]
    data['SMD410'] = (data['SMD410'] == 1).astype(int)


    data['SH EXP']  = data['SMD460'] + data['SMD410']
    data = data.drop(['SMD410','SMD460'], axis = 1)

    #1 if the person smokes
    data['SMOKE'] = (data['SMQ040']<2).astype(int)

    return data.drop(['SMQ040'], axis = 1)

def process(df, transform = False, combine_columns = True):

    """This applies a log transformation to the predictors and combines the response variables int0 a single response
    transform: if the log transformation should be applied
    combine_columns: if the responses should be combines: this should only be false if you want to partition the data by response groups.
    """

    data = df.copy()
    data = relable(data)

    #people who are not exposed to tobacco at all
    non_smokers = data[data['SMOKE'] == 0]
    non_smokers = non_smokers[non_smokers['SH EXP'] == 0]
    #people exposed to smoke, but who do not smoke
    sh_smokers = data[data['SMOKE'] == 0]
    sh_smokers = sh_smokers[sh_smokers['SH EXP'] == 1]

    smokers = data[data['SMOKE'] == 1]

    data = pd.concat([non_smokers, sh_smokers, smokers])

    if combine_columns: # make a single response variable: 0,1,2 
            data['SMOKE'] = data['SMOKE']+ data['SH EXP']
            data = data.drop(['SH EXP'], axis = 1)

    #take the log transformation
    if transform == True:
        data['LBXVBZ'] = np.log(data['LBXVBZ'])
        data['LBXVEB'] = np.log(data['LBXVEB'])
        data['LBXVXY'] = np.log(data['LBXVXY'])
    
    #reshuffle
    return data.sample(frac=1).reset_index(drop=True)

In [ ]:
predictors_list = ['LBXVBZ','LBXVEB','LBXVXY']
response_var_d = 'SMQ040'
response_var_sh_2013 = 'SMD410'
response_var_sh_2014 = 'SMD460'

#transformed and seperate responses (i.e. one columns for secod hand exposure, and one columns for whether or not they smoke)
nhanes_log_sc= process(fetch_data(urls_2013, urls_2014, predictors_list, 
                                              response_var_d, response_var_sh_2013, 
                                              response_var_sh_2014, save = False), 
                                              transform= True, combine_columns=False)
#transformed with combined responses : this is the frame that gets used for prediction
nhanes_log__cc = process(fetch_data(urls_2013, urls_2014, predictors_list, 
                                              response_var_d, response_var_sh_2013, 
                                              response_var_sh_2014, save = False), 
                                              transform= True, combine_columns=True)
#untransfored and seperate repsonses
nhanes_no_log_sc = process(fetch_data(urls_2013, urls_2014, predictors_list, 
                                              response_var_d, response_var_sh_2013, 
                                              response_var_sh_2014, save = False), 
                                              transform= False, combine_columns=False)
#untransformed with combined responses.
nhanes_no_log_cc = process(fetch_data(urls_2013, urls_2014, predictors_list, 
                                              response_var_d, response_var_sh_2013, 
                                              response_var_sh_2014, save = False), 
                                              transform= False, combine_columns=True)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(nhanes_log__cc.drop(['SMOKE'], axis = 1), nhanes_log__cc['SMOKE'])

pipe = Pipeline(steps = [('scale',StandardScaler()), ('clf', DecisionTreeClassifier())])

param_grid = {
    'clf__criterion': ['gini', 'entropy'],
    'clf__max_depth': [None, 5, 10],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4]
}

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, verbose=3)
pipe.fit(X_train[:200], y_train[:200])
pipe.score(X_test, y_test)
grid.fit(X_train, y_train)
dt_mod = grid.best_estimator_
print(classification_report(dt_mod.predict(X_test), y_test))
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, dt_mod.predict(X_test), labels=dt_mod.classes_),
                              display_labels=dt_mod.classes_)
disp.plot()
plt.title('Classification Tree Confusion Matrix')
plt.show()

In [ ]:
rf = dt_mod.named_steps['clf']

new_cols = ['Benzene', 'Ethylbenzene', 'Xylene']

plt.barh(new_cols, 
        width = np.sort(rf.feature_importances_),
        color = ["#F9EAF9","#E1C2E1","#C8AAC8","#AB90AB","#917A91","#6C596C"], 
        edgecolor = "#413E41")
plt.title("Classification Tree Feature Importance Plot")
plt.xlabel("Avg. Decrease in Impurity")
plt.ylabel("Feature")
plt.show()